# Unsupervised Learning and Semi-Supervised Models
Working with unlabeled and semi-labeled patient data since we cannot assume most clients have chosen a desired clinic to frequent.

In [0]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import scipy as sp
%matplotlib inline

from collections import Counter

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN

from sklearn.decomposition import PCA

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV

# Keras neural net imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.convolutional import Convolution1D

# libraries for boosting
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Preprocessing
Make sure data are in the directory

In [0]:
# Load training and test sets in current working directory
x_train = pd.read_csv("x_train.csv").drop('Unnamed: 0', axis=1)
# get rid of unnamed first column and unravel (n,1) into dim (n,)
y_train = pd.read_csv("y_train.csv").drop('Unnamed: 0', axis=1).values.ravel()
x_test = pd.read_csv("x_test.csv").drop('Unnamed: 0', axis=1)
y_test = pd.read_csv("y_test.csv").drop('Unnamed: 0', axis=1).values.ravel()

In [0]:
x_combined = pd.concat([x_train,x_test], axis=0)

In [0]:
f_x_train = pd.read_csv("filtered_x_train.csv").drop('Unnamed: 0', axis=1)
f_x_test = pd.read_csv("filtered_x_test.csv").drop('Unnamed: 0', axis=1)

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since):
    now = time.time()
    s = now - since
    return 'Time elapsed: %s (%s to %s)' % (asMinutes(s), asMinutes(since), asMinutes(now))

In [0]:
def scoreRMSE(predictor, X, true_y, plot = True):
    '''
    function to compute root mean squared error as well as prediction accuracy

    @args:
        predictor: model
        X: test input data
        true_y: true y labels for test data
        plot: only for boosting, plots f-scores for each booster
    '''
    predictions = predictor.predict(X)
    print("\nModel Report")
    print ("RMSE Score : %f" % np.sqrt(mean_squared_error(predictions, true_y)))
    accuracy = metrics.accuracy_score(predictions, true_y) * 100
    print("Accuracy : %.4g" % accuracy, '%')         

    if plot:
        # Print f-score:    
        feat_imp = pd.Series(predictor.get_booster().get_fscore()).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='f-score', figsize = (20, 10))
        plt.ylabel('f-score')

## Semisupervised: PCA
Perform PCA on labelled and unlabelled data. Use the fitted PCA to transform labelled data, which is used as input for training.

In [0]:
n_components = 8
pca = PCA(n_components=n_components, whiten=False)

start = time.time()

# fit using all input data
pca.fit(x_combined)

# generate transformed projections for combined x, x_train, and x_test
x_pca = pca.transform(x_train)
x_train_pca = pd.DataFrame(x_pca)
x_pca = pca.transform(x_test)
x_test_pca = pd.DataFrame(x_pca)
x_pca = pca.transform(x_combined)
x_combined_pca = pd.DataFrame(x_pca)

In [0]:
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)')
plt.title('Dataset Explained Variance')
plt.show()

### Supervised Training on PCA projections

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


# Number of trees in random forest
n_estimators = n_estimators = list(range(10, 100, 10))

# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = list(range(10, 30, 10))
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [0]:
print ("learning...")
rfc = RandomForestClassifier(min_samples_split=10)
rf_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(x_train_pca, y_train)
print(rf_random.best_params_) #{'bootstrap': True, 'max_features': 'sqrt', 'min_samples_split': 2, 'max_depth': 30, 'n_estimators': 300, 'min_samples_leaf': 2}
print("\nTraining Results")
scoreRMSE(rf_random, x_train_pca, y_train, plot = False) 
print("\nEvaluation Results")
scoreRMSE(rf_random, x_test_pca, y_test, plot = False)
print ("done learning")

## Active Learning/Pseudo-labeling
Train supervised model on labelled data to provide pseudo-labels for unlabelled data. Then train a fresh model on the combined labelled and pseudo-labeled data

In [0]:
# we build off of our PCA work and use pca data

# pseudo labels predicted by model
y_test_ps = rf_random.predict(x_test_pca)

# concat labelled and model-labelled data
y_combined_ps = np.concatenate([y_train,y_test_ps], axis=0)

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


# Number of trees in random forest
n_estimators = n_estimators = list(range(10, 100, 10))

# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = list(range(10, 30, 10))
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [0]:
print ("learning...")
rfc = RandomForestClassifier(min_samples_split=10)
rf_rand_ps = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

rf_rand_ps.fit(x_combined_pca, y_combined_ps)
print(rf_rand_ps.best_params_) #{'bootstrap': True, 'max_features': 'sqrt', 'min_samples_split': 2, 'max_depth': 30, 'n_estimators': 300, 'min_samples_leaf': 2}
print("\nTraining Results")
scoreRMSE(rf_rand_ps, x_train_pca, y_train, plot = False) 
print("\nEvaluation Results")
scoreRMSE(rf_rand_ps, x_test_pca, y_test, plot = False)
print ("done learning")

## Unsupervised: Clustering

In [0]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

print(x_combined)

max_cluster_count = 4
for k in range(3, max_cluster_count):
  km = KMeans(
    n_clusters=k, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
  )
  y_km = km.fit_predict(x_combined)




[[ 2.60509732  1.22529553]
 [ 0.5323772   3.31338909]
 [ 0.802314    4.38196181]
 [ 0.5285368   4.49723858]
 [ 2.61858548  0.35769791]
 [ 1.59141542  4.90497725]
 [ 1.74265969  5.03846671]
 [ 2.37533328  0.08918564]
 [-2.12133364  2.66447408]
 [ 1.72039618  5.25173192]
 [ 3.1368855   1.56592763]
 [-0.37494566  2.38787435]
 [-1.84562253  2.71924635]
 [ 0.72144399  4.08475018]
 [ 0.16117091  4.53517846]
 [-1.99912714  2.71285741]
 [-1.47804153  3.2093591 ]
 [ 1.8706766   0.77797407]
 [-1.5933443   2.76898682]
 [ 2.03562611  0.31361691]
 [ 0.64003985  4.12401075]
 [ 2.4411628   1.30941574]
 [ 1.13280393  3.87673946]
 [ 1.04829186  5.03092408]
 [-1.26637157  2.62998828]
 [ 2.31690585  0.81189049]
 [ 2.36230721  1.358767  ]
 [ 1.2091013   3.53566548]
 [-2.54224625  3.95012869]
 [ 1.4815332   0.67875364]
 [-1.59487886  3.48632794]
 [-1.82556205  2.7989214 ]
 [-1.13374003  2.68467271]
 [-1.758702    3.158623  ]
 [ 0.3498724   4.69253251]
 [ 1.68548602  1.66917096]
 [ 2.989047    1.35068599]
 